# TPMS with SDF and PyScaffolder
Author: [Jirawat Iamsamang](https://github.com/nodtem66)  

## Abstract
SDF provides a class for discretizing and visualizing any implicit surfaces. The basic topologies (e.g. sphere, box) are already defined.
This notebook shows how to utilize this library to generate gyroid surface.

## Installation
* Currently, SDF is not in PyPI. So the [github of SDF](https://github.com/fogleman/sdf) needs to clone into local computer. See [Installation](https://github.com/fogleman/sdf#installation)
* PyScaffolder can be installed by `pip install PyScaffolder`


In [ ]:
!git clone https://github.com/fogleman/sdf
!pip install PyScaffolder

Cloning into 'sdf'...
remote: Enumerating objects: 861, done.
remote: Counting objects: 100% (848/848), done.
remote: Compressing objects: 100% (409/409), done.
remote: Total 861 (delta 448), reused 818 (delta 433), pack-reused 13
Receiving objects: 100% (861/861), 8.26 MiB | 8.67 MiB/s, done.
Resolving deltas: 100% (448/448), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.4/786.4 kB 11.5 MB/s eta 0:00:00


## The implicit function of TPMS

Name	| F(x,y,z) 
--- | --- 
Schwarz-P	| cos⁡(x)+cos⁡(y)+cos⁡(z)
Double Schwarz-P |	cos⁡(x)  cos⁡(y)+  cos⁡(y)  cos⁡(z)+ cos⁡(x)  cos⁡(z)+  0.35 [cos⁡(2x)+cos⁡(2y)+cos⁡(2z)]	
Diamond |	sin⁡(x)  sin⁡(y)  sin⁡(z)+sin⁡(x)  cos⁡(y)  cos⁡(z) + cos⁡(x)  sin⁡(y)  cos⁡(z)+  cos⁡(x)  cos⁡(y)  sin⁡(z)	
Double Diamond | sin⁡(2x)  sin⁡(2y)+ sin⁡(2y)  sin⁡(2z) + sin⁡(2z)  sin⁡(2x)+  cos⁡(2x)  cos⁡(2y)  cos⁡(2z)	
Gyroid|	cos⁡(x)  sin⁡(y)+cos⁡(y)  sin⁡(x)+cos⁡(z)  sin⁡(x)	
Double Gyroid |	2.75 [ sin(2x)  sin⁡(z)  cos⁡(y)+sin⁡(2y)  sin⁡(x)  cos⁡(z) + sin⁡(2z)  sin⁡(y) cos⁡(x) ] - [ cos⁡(2x)cos⁡(2y)+ cos⁡(2y)  cos⁡(2z)+cos⁡(2z) cos⁡(2x)  ]	
Lidinoid |	sin⁡(2x)  cos⁡(y)  sin⁡(z)+sin⁡(2y)  cos⁡(z)  sin⁡(x)+ sin⁡(2z)  cos⁡(x)  sin⁡(y)+ cos⁡(2x)  cos⁡(2y)+ cos⁡(2y)  cos⁡(2z)+cos⁡(2z)  cos⁡(2x)	
Neovius |	3 [cos⁡(x)+cos⁡(y)+cos⁡(z) ]+4 [cos⁡(x)  cos⁡(y)  cos⁡(z) ]	
Schoen IWP |	cos⁡(x)  cos⁡(y)+cos⁡(y)  cos⁡(z)+cos⁡(z)  cos⁡(x)	
Tubular G AB |	20 [cos⁡(x)  sin⁡(y) + cos⁡(y)  sin⁡(z)+cos⁡(z)  sin⁡(x)  ] -0.5 [cos⁡(2x)  cos⁡(2y)+cos⁡(2y)  cos⁡(2z)+cos⁡(2z)  cos⁡(2x) ] -4 	
Tubular G C	| -10 [cos⁡(x)  sin⁡(y)+cos⁡(y)  sin⁡(z)+cos⁡(z)  sin⁡(x) ] +2[cos⁡(2x)  cos⁡(2y)+cos⁡(2y)  cos⁡(2z)+cos⁡(2z)  cos⁡(2x)  ] +12 	
BCC |	cos⁡(x)+cos⁡(y)+cos⁡(z)-2 [ cos⁡(x/2)cos⁡(y/2) + cos⁡(y/2)cos⁡(z/2)+cos⁡(z/2)cos⁡(x/2) ]	


## Gyroid
The gyroid function is defined as shown in a following cell.  
The wrapper `@sdf3` will provide gyroid function with 3D points (`p`)).
Then these (x,y, z) points will multiply by `w` and calculate the iso-level of gyroid by vectorized numpy function. 



In [ ]:
%cd /content/sdf
%load_ext autoreload
%autoreload 2
# Add parent directory into system path
import sys, os
sys.path.insert(1, os.path.abspath(os.path.normpath('..')))

import numpy as np
from sdf import *

@sdf3
def gyroid(w = 3.14159, t=0):
    def f(p):
        q = w*p
        x, y, z = (q[:, i] for i in range(3))
        return np.cos(x)*np.sin(y) + np.cos(y)*np.sin(z) + np.cos(z)*np.sin(x) - t
    return f

/content/sdf
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Generate with SKimage
SDF used `marching_cubes` from `skimage.measure` with a `ThreadPool`, so it's super fast to construct the 3D mesh.
Let's create a constructing function that intersect a gyroid and a unit box.

In [ ]:
f = box(1) & gyroid(w=12)

In [ ]:
# Generate with skimage.measure.marching_cubes
points = f.generate(step=0.01, verbose=True)
write_binary_stl('out_1.stl', points)

min -0.543491, -0.543491, -0.543491
max 0.543491, 0.543491, 0.543491
step 0.01, 0.01, 0.01
1404928 samples in 64 batches with 2 workers

6 skipped, 0 empty, 58 nonempty
239438 triangles in 1.04591 seconds


## Generate with PyScaffolder

However, this method occasionally results in incomplete mesh.  
Then let's try `Pyscaffolder.marching_cubes` which implements `dual marching cubes` from [@dominikwodniok/dualmc](https://github.com/dominikwodniok/dualmc).

In [ ]:
# Generate with PyScaffolder.marching_cubes
import PyScaffolder
def marching_cubes(f, step=0.01, bounds=None, verbose=True, clean=True):
    from sdf.mesh import _estimate_bounds, _cartesian_product
    import time

    if not bounds:
        bounds = _estimate_bounds(f)
    (x0, y0, z0), (x1, y1, z1) = bounds
    
    try:
        dx, dy, dz = step
    except TypeError:
        dx = dy = dz = step
    
    if verbose:
        print('min %g, %g, %g' % (x0, y0, z0))
        print('max %g, %g, %g' % (x1, y1, z1))
        print('step %g, %g, %g' % (dx, dy, dz))

    X = np.arange(x0, x1, dx)
    Y = np.arange(y0, y1, dy)
    Z = np.arange(z0, z1, dz)

    P = _cartesian_product(X, Y, Z)
    try:
        # Since the PyScaffolder marching_cubes aceept FREP: F(x,y,z) > 0
        # Then the negative of implicit function is used
        Fxyz = (-f(P))
        # Reshape to Fortran array (column-based) due to implementation of dualmc starting from z axis to x
        Fxyz = Fxyz.reshape((len(X), len(Y), len(Z))).reshape(-1, order='F')
        start = time.time()
        (v, f) = PyScaffolder.marching_cubes(Fxyz, grid_size=[len(X), len(Y), len(Z)], v_min=bounds[0], delta=step, clean=clean)
        if verbose:
            seconds = time.time() - start
            print('\n%d triangles in %g seconds' % (len(f) // 3, seconds))
        # merge vertices and faces into points
        return v[f].reshape((-1, 3))
    except Exception as e:
        print(e)
        return np.array([])

points = marching_cubes(f, step=0.01, verbose=True, clean=True)
write_binary_stl('out_2.stl', points)

min -0.543491, -0.543491, -0.543491
max 0.543491, 0.543491, 0.543491
step 0.01, 0.01, 0.01

82748 triangles in 1.07692 seconds


## Generate the other TPMS

The following codes will generate SchwarzP and BCC structure. The output files can be found at `/content/sdf/` 

In [ ]:
@sdf3
def schwarz_p(w = 3.14159, t=0):
    def f(p):
        q = w*p
        x, y, z = (q[:, i] for i in range(3))
        return np.cos(x) + np.cos(y) + np.cos(z)
    return f

@sdf3
def bcc(w = 3.14159, t=0):
    def f(p):
        q = w*p
        x, y, z = (q[:, i] for i in range(3))
        return np.cos(x) + np.cos(y) + np.cos(z) -2*(np.cos(x/2)*np.cos(y/2) + np.cos(y/2)*np.cos(z/2) + np.cos(z/2)*np.cos(x/2))
    return f

In [ ]:
f2 = box(1) & schwarz_p(w=12)
points = marching_cubes(f2, step=0.01, verbose=True, clean=True)
write_binary_stl('schwarz_p.stl', points)

min -0.543491, -0.543491, -0.543491
max 0.543491, 0.543491, 0.543491
step 0.01, 0.01, 0.01

59388 triangles in 0.45409 seconds


In [ ]:
f3 = box(1) & bcc(w=12)
points = marching_cubes(f3, step=0.01, verbose=True, clean=True)
write_binary_stl('bcc.stl', points)

min -0.543491, -0.543491, -0.543491
max 0.543491, 0.543491, 0.543491
step 0.01, 0.01, 0.01

49112 triangles in 0.377038 seconds
